<a href="https://colab.research.google.com/github/tapasML/EPAi2_Session7/blob/main/Session_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment 1**

Intall Faker and import packages

In [1]:
try:
  import faker
except:
  !pip install Faker
  import faker
  
from faker import Faker
from  collections import namedtuple
from decimal import Decimal
import datetime
from datetime import date
import time

utility function to calculate age from date

In [2]:
def calculate_age(dob: date):

  ''' calculate age in years from dob'''

  today = date.today()
  diff = today - dob
  return diff.days/(365)

# Using namedtuple

In [3]:
def calculate_with_namedtuple()-> tuple:

  ''' Using namedtuple:  Use Faker library to get 10000 random profiles. 
  Using namedtuple, calculate the largest blood type, mean-current_location, 
  oldest_person_age and average age'''

  _start = time.perf_counter()
  faker = Faker() 
  # generate a profile
  profile_dict=faker.profile()  

  # get the attribute names
  keys=tuple(profile_dict.keys())   
  person_list = []
  person = namedtuple('Profile', keys)
  
  for i in range(10000):      
    # p = person(*faker.profile())    
    p = person(*faker.profile().values())  
    person_list.append(p)

  # largest blood type
  _max_blood_group = max([item.blood_group for item in person_list])

  # mean-current_location
  _len = len(profile_dict)
  _mean_x_coord = sum([item.current_location[0] for item in person_list]) / _len
  _mean_y_coord = sum([item.current_location[1] for item in person_list]) / _len
  _mean_location = _mean_x_coord, _mean_y_coord

  # oldest_person_age
  _max_age = calculate_age(min([item.birthdate for item in person_list]))

  # average age
  _average_age = sum([calculate_age(item.birthdate) for item in person_list]) / _len

  _end = time.perf_counter()
  
  return _max_blood_group, _mean_location, _max_age, _average_age, _end - _start 

Test using namedtuple

In [4]:
namedtuple_time = (calculate_with_namedtuple())
print(namedtuple_time[4])
print('time taken = ', namedtuple_time)

13.023049533999256
time taken =  ('O-', (Decimal('-727.9370096153846153846153846'), Decimal('-344.6622967692307692307692308')), 116.06849315068493, 44240.482402529015, 13.023049533999256)


# Using dictionary

In [5]:
def calculate_with_dict():

  ''' Using dictionary : Use Faker library to get 10000 random profiles. 
  Using namedtuple, calculate the largest blood type, mean-current_location, 
  oldest_person_age and average age'''

  _start = time.perf_counter()
  faker = Faker() 
  # generate a profile
  profile_dict=faker.profile()  

  # get the attribute names  
  keys=list(profile_dict.keys()) 
  person_list = []  
  
     
  for i in range(10000): 
    _dict={} 
    profile = faker.profile()
    for item in keys: 
      _dict[item] = profile[item]     
    person_list.append(_dict)

  # largest blood type
  _max_blood_group = max([item['blood_group'] for item in person_list])

  # mean-current_location
  _len = len(profile_dict)
  _mean_x_coord = sum([item['current_location'][0] for item in person_list]) / _len
  _mean_y_coord = sum([item['current_location'][1] for item in person_list]) / _len
  _mean_location = _mean_x_coord, _mean_y_coord

  # oldest_person_age
  _max_age = calculate_age(min([item['birthdate'] for item in person_list]))

  # average age
  _average_age = sum([calculate_age(item['birthdate']) for item in person_list]) / _len
  
  _end = time.perf_counter()
  return _max_blood_group, _mean_location, _max_age, _average_age, _end - _start 


Test using dictionary

In [6]:
dict_time = calculate_with_dict()
print(dict_time)
print(dict_time[4])

('O-', (Decimal('211.6559145'), Decimal('493.4697038461538461538461538')), 116.05479452054794, 44546.71654373005, 12.898824904999856)
12.898824904999856


# Stock Market Assignment Problem

---

In [7]:
import random

num_companies = 100
num_instances = 50

# 50 time instances of a day. Staring 08:00 a.m , every 10 minutes
times_of_day = ['min_'+str(t) for t in range(480, 980, 10)]

# declare the namedtuple
company_stock=namedtuple('Company', ('c_name', 'c_symbol', 'c_weight', *times_of_day))


def create_stock_weight():

  ''' create 100 normalized random stock weights'''

  weights=[]
  for i in range(num_companies):
    weights.append(random.randint(1,100))
  total = sum(weights)  
  weights = [round(wt/total, 4) for wt in weights]
  return weights
  

# create normalized weights for 100 stocks
weights = create_stock_weight()

def create_symbol():

  ''' create 1 fake company name and symbol using faker''' 
  
  faker = Faker()
  name = faker.company()    
  symbol = str.upper(name[:4])    
  return name, symbol

def createOneCompanyRecord(i): 

  ''' create one company stock value record for a day''' 

  stock_weight = weights[i]  
  opening_value = random.randint(1,1000) * stock_weight  
  stock_values = [round(opening_value * random.uniform(0.90, 1.10), 4) for a in range(num_instances)]  
  c_stock = company_stock(*create_symbol(), stock_weight, *stock_values)
  return c_stock

# Create Stocks for 100 companies at 50 instances in day

In [8]:
def createCompanyStockRecords(): 

  ''' create daily stock records for 100 companies'''
 
  stock_records=[]
  for i in range(num_companies):
    stock_records.append(createOneCompanyRecord(i))
  return stock_records
  

# Analyze stock records

In [9]:
def calculate_value_specific_instance(stock_records, i):

  ''' calculate/generate total stock market value at specific instance in the day'''

  time_instance = times_of_day[i] 
  open_value = round(sum([getattr(rec,times_of_day[i]) for rec in stock_records]), 4)  
  return open_value


def calculate_max_and_time(stock_records)->tuple:

  ''' Find the manixum stock value and at what time in day. For instance 830 indicates 830 minutes since midnight '''

  stock_values =[]
  for _time in times_of_day:
    stock_values.append(round(sum([getattr(rec,_time) for rec in stock_records]), 4))  

  max_index = stock_values.index(max(stock_values))
  max_value = stock_values[max_index]
  max_time = str((times_of_day[max_index])[4:]) + ' minutes'
  return max_value, max_time

# Test stock records

In [10]:
# create stock records
stock_records = createCompanyStockRecords()

open_val = calculate_value_specific_instance(stock_records, 0) # opening value

close_val = calculate_value_specific_instance(stock_records, 49) # closing value

max_val, max_at_time = calculate_max_and_time(stock_records)   # max value and time

print('Stock Market open value = ', open_val)
print('Stock Market close value = ', close_val)
print('Stock Market max value @ time (minutes) = ', max_val, max_at_time)


Stock Market open value =  491.1279
Stock Market close value =  496.0231
Stock Market max value @ time (minutes) =  504.4648 880 minutes


# **Assignment 2**
# Polygon **1** : 
Test Happy Path

---



In [11]:
from polygon import *
import sys

poly1= Polygon(6, 10) 
poly2 = Polygon(6, 10)
poly3 = Polygon(8, 7)

print(poly1) # test __repr__() function
print('\npoly1 == poly2 :',poly1 == poly2) # test __eq__() 
print('invalid comparison :',poly1 == 100) # test __eq__() 
print('poly3 > poly2 :', poly3 > poly2) # test __gt__()

Polygon(
numedges=6,
radius=10,
interior_angle=120.0,          
edge_length=10.0,
apothem=8.6603,
area=259.809,
perimeter=60.0)

poly1 == poly2 : True
invalid comparison : False
poly3 > poly2 : True


Test Boundary Conditions

In [12]:
try:
  poly1= Polygon(2, 10) # invalid number of edges
except ValueError as e:
  print(e)

Polygon: number of vertices must be >= 3


In [13]:
try:
  cp1 = Polygon(25,-10)
except ValueError as e:
  print(e)

Polygon: radius must be > 0


# CustomPolygon 

---
Test Happy Path

In [14]:
cp1 = CustomPolygon(5,6)
print(cp1)

polygon sequence [Polygon(
numedges=3,
radius=6,
interior_angle=60.0,          
edge_length=10.3923,
apothem=3.0,
area=46.7654,
perimeter=31.1769), Polygon(
numedges=4,
radius=6,
interior_angle=90.0,          
edge_length=8.4853,
apothem=4.2426,
area=71.9995,
perimeter=33.9412), Polygon(
numedges=5,
radius=6,
interior_angle=108.0,          
edge_length=7.0534,
apothem=4.8541,
area=85.5948,
perimeter=35.267)]


In [15]:
print(cp1[0])
print()
print(len(cp1))
print()
print(cp1._max_efficient)

Polygon(
numedges=3,
radius=6,
interior_angle=60.0,          
edge_length=10.3923,
apothem=3.0,
area=46.7654,
perimeter=31.1769)

3

Polygon(
numedges=5,
radius=6,
interior_angle=108.0,          
edge_length=7.0534,
apothem=4.8541,
area=85.5948,
perimeter=35.267)


Test Boundary Conditions

In [16]:
try:
  cp1 = CustomPolygon(2,6)
except ValueError as e:
  print(e)

CustomPolygon: number of vertices must be >= 3


In [17]:
try:
  cp1 = CustomPolygon(10,6)
  cp1[11]
except ValueError as e:
  print(e)

CustomPolygon: invalid key
